In [13]:
import datetime as dt
import numpy as np
import pandas as pd

In [14]:
from Curves import Curves
from ImportData import importSWEiopa
from EquityClasses import Equity, EquityPriced

In [15]:
paramfile = pd.read_csv("Input/Parameters.csv")
paramfile.index = paramfile["Parameter"]
del paramfile["Parameter"]

In [16]:
spreadfile = pd.read_csv("Input/Sector_Spread.csv")
spreadfile.index = spreadfile["NACE"]
del spreadfile["NACE"]

# IMPORT EIOPA CURVE

In [17]:
selected_param_file = paramfile.loc["EIOPA_param_file"][0]
selected_curves_file = paramfile.loc["EIOPA_curves_file"][0]
country = paramfile.loc["country"]["value"]
compounding = int(paramfile.loc["compounding"]["value"])

# Model parameters
MD = dt.datetime.strptime(paramfile.loc["Modelling_Date"]["value"],"%d/%m/%Y")
MD = dt.date(MD.year,MD.month,MD.day)

[maturities_country, curve_country, extra_param, Qb]= importSWEiopa(selected_param_file, selected_curves_file, country)

In [18]:
# Maturity of observations:
M_Obs = np.transpose(np.array(maturities_country.values))

# Ultimate froward rate ufr represents the rate to which the rate curve will converge as time increases:
ufr = extra_param.iloc[3]/100

# Convergence speed parameter alpha controls the speed at which the curve converges towards the ufr from the last liquid point:
alpha = extra_param.iloc[4]

# Qb calibration vector published by EIOPA for the curve calibration:
Qb = np.transpose(np.array(Qb.values))

# PRICING EQUITY

Equity object contains all the information necessary to identify and project the cashflows.

Simulate cashflows from dividend payments from the modelling date until the end of the simulation period.

Calibrate the growth rate

$$
MV = \frac{MV*(1+g) dy}{1+y_1}+\frac{MV*(1+g)^2 dy}{(1+y_2)^2}+\dots + \frac{MV*(1+g)^m dy}{(1+y_m)^m} + \frac{1}{(1+y_m)^m} \frac{MV*dy}{r-g}
$$

$$
1 = \frac{(1+g) dy}{1+y_1}+\frac{(1+g)^2 dy}{(1+y_2)^2}+\dots + \frac{(1+g)^m dy}{(1+y_m)^m} + \frac{1}{(1+y_m)^m} \frac{dy}{r-g}
$$

$$
\frac{1}{dy} = \frac{(1+g)}{1+y_1}+\frac{(1+g)^2}{(1+y_2)^2}+\dots + \frac{(1+g)^m}{(1+y_m)^m} + \frac{1}{(1+y_m)^m} \frac{1}{r-g}
$$





In [19]:
equityfile = pd.read_csv("Input/Equity_Portfolio.csv")
equityfile.index = equityfile["Asset ID"]
del equityfile["Asset ID"]
equityfile

,Asset_Type,Issuer_Name,NACE,Issue_Date,Dividend_Yield,Frequency,Market_Price
Asset ID,,,,,,,
1,Equity,Open Source Modelling,A1.4.5,3/12/2021,0.03,1,94
2,Equity,Open Source Modelling,B5.2.0,3/12/2021,0.05,1,92
3,Equity,Open Source Modelling,B8.9.3,3/12/2019,0.04,1,96


In [20]:
def converttime(date_time):
    format = "%d/%m/%Y"
    datetime_str = dt.datetime.strptime(date_time,format).date()
    return datetime_str

In [21]:
nace = np.array([])
issuername = np.array([])
issuedate = np.array([])
dividendyield = np.array([])
frequency = np.array([])
marketprice = np.array([])

for iCount in range(0,equityfile.index.size):
    nace = np.append(nace,equityfile["NACE"].iloc[iCount])
    issuedate = np.append(issuedate,converttime(equityfile["Issue_Date"].iloc[iCount]))
    issuername = np.append(issuername, equityfile["Issuer_Name"].iloc[iCount])
    dividendyield = np.append(dividendyield,equityfile["Dividend_Yield"].iloc[iCount])
    frequency = np.append(frequency,equityfile["Frequency"].iloc[iCount])
    marketprice = np.append(marketprice, equityfile["Market_Price"].iloc[iCount])

In [22]:
zcb = Equity(nace, issuedate, issuername, dividendyield, frequency, marketprice)

The createcashflows function does the folowing steps:

 - 1) Calculates the fractions of dates for every dividend payout from modelling date until the end of the modelling period
 - 2) Calculate the yield at each date the dividend is paid
 - 3) Calibrate the growth rate of the equity using the formula above
 - 4) Use growth rate to calculate the evolution of Market Value
 - 5) Use market value to calculate the size of each dividend cash flow

In [23]:
#zcb.createcashflows()
enddate = dt.date(2073,4,29)

In [24]:
zcbPriced = EquityPriced(MD,compounding, enddate)

In [ ]:
EquityPriced.createcashflows() # ToDo

In [25]:
[coupondatesconsidered,coupondatefrac] = zcbPriced.refactordates(zcb.coupondates,MD)
[notionaldatesconsidered,notionaldatefrac] = zcbPriced.refactordates(zcb.notionaldates,MD)

AttributeError: 'EquityPriced' object has no attribute 'refactordates'

In [ ]:
# carry forward the sspread of bonds that are still relevant
zcbPriced.sspread = []
zcbPriced.marketprice = []

for iCount in range(0,len(notionaldatesconsidered)-1):
    if notionaldatesconsidered[iCount] == [0]:
        zcbPriced.sspread = np.append(zcbPriced.sspread,zcb.sspread[iCount])
        zcbPriced.marketprice = np.append(zcbPriced.marketprice,zcb.marketprice[iCount])

In [ ]:
# Numeric precision of the optimisation
Precision = float(paramfile.loc["Precision"][0])
# Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point
Tau = float(paramfile.loc["Tau"][0]) # 1 basis point
Country = paramfile.loc["country"]
InitialDate = paramfile.loc["Modelling_Date"]
Curves = Curves(ufr, Precision, Tau, InitialDate, Country)

In [ ]:
coupontargetrates = []
notionaltargetrates = []

for iAsset in range(0,issuedate.size-1):

    if not notionaldatesconsidered:
       print("This bond has matured") 
    else: 
        coupontargetrates.append(Curves.SWExtrapolate(np.transpose(coupondatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        notionaltargetrates.append(Curves.SWExtrapolate(np.transpose(notionaldatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        zcbPriced.coupondatefrac.append(np.transpose(coupondatefrac[iAsset]))
        zcbPriced.notionaldatefrac.append(np.transpose(notionaldatefrac[iAsset]))
        zcbPriced.couponcfs.append(zcb.couponcfs[iAsset][coupondatesconsidered[iAsset]])
        zcbPriced.notionalcfs.append(zcb.notionalcfs[iAsset])

Remove cashflows that are not considered

In [ ]:
#zcbPriced.PriceBond(coupontargetrates,notionaltargetrates,coupondatefrac,notionaldatefrac,zcbPriced.couponcfs,zcbPriced.notionalcfs,zcbPriced.sspread,np.array([0.01,0.01]))

### Calibration of two hypothetical bonds that mature after modelling date

In [ ]:
def BisectionsSpread(zcbPriced, xStart, xEnd, coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread, marketprice, Precision, maxIter):
    """
    Bisection root finding algorithm for finding the root of a function. The function here is the allowed difference between the ultimate forward rate and the extrapolated curve using Smith & Wilson.

    Args:
        xStart =    1 x 1 floating number representing the minimum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.05
        xEnd =      1 x 1 floating number representing the maximum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.8
        M_Obs =     n x 1 ndarray of maturities of bonds, that have rates provided in input (r). Ex. u = [[1], [3]]
        r_Obs =     n x 1 ndarray of rates, for which you wish to calibrate the algorithm. Each rate belongs to an observable Zero-Coupon Bond with a known maturity. Ex. r = [[0.0024], [0.0034]]
        ufr  =      1 x 1 floating number, representing the ultimate forward rate. Ex. ufr = 0.042
        Tau =       1 x 1 floating number representing the allowed difference between ufr and actual curve. Ex. Tau = 0.00001
        Precision = 1 x 1 floating number representing the precision of the calculation. Higher the precision, more accurate the estimation of the root
        maxIter =   1 x 1 positive integer representing the maximum number of iterations allowed. This is to prevent an infinite loop in case the method does not converge to a solution         
    
    Returns:
        1 x 1 floating number representing the optimal value of the parameter alpha 

    Example of use:
        >>> import numpy as np
        >>> from SWCalibrate import SWCalibrate as SWCalibrate
        >>> M_Obs = np.transpose(np.array([1, 2, 4, 5, 6, 7]))
        >>> r_Obs =  np.transpose(np.array([0.01, 0.02, 0.03, 0.032, 0.035, 0.04]))
        >>> xStart = 0.05
        >>> xEnd = 0.5
        >>> maxIter = 1000
        >>> alfa = 0.15
        >>> ufr = 0.042
        >>> Precision = 0.0000000001
        >>> Tau = 0.0001
        >>> BisectionAlpha(xStart, xEnd, M_Obs, r_Obs, ufr, Tau, Precision, maxIter)
        [Out] 0.11549789285636511

    For more information see https://www.eiopa.europa.eu/sites/default/files/risk_free_interest_rate/12092019-technical_documentation.pdf and https://en.wikipedia.org/wiki/Bisection_method
    
    Implemented by Gregor Fabjan from Qnity Consultants on 17/12/2021.
    """   

    yStart = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xStart)
    yEnd = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xEnd)
    if np.abs(yStart-marketprice) < Precision:
        #self.alpha = xStart # If initial point already satisfies the conditions return start point
        return xStart
    if np.abs(yEnd-marketprice) < Precision:
        #self.alpha = xEnd
        return xEnd # If final point already satisfies the conditions return end point
    iIter = 0
    while iIter <= maxIter:
        xMid = (xEnd+xStart)/2 # calculate mid-point
        yMid = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xMid) # What is the solution at midpoint
        if ((yStart-marketprice) == 0 or (xEnd-xStart)/2 < Precision): # Solution found
            #self.alpha = xMid
            return xMid
        else: # Solution not found
            iIter += 1
            if np.sign(yMid-marketprice) == np.sign(yStart-marketprice): # If the start point and the middle point have the same sign, then the root must be in the second half of the interval   
                xStart = xMid
            else: # If the start point and the middle point have a different sign than by mean value theorem the interval must contain at least one root
                xEnd = xMid
    return "Did not converge"

In [ ]:
zcbPriced.zspread = []
for iCount in range(0,len(coupontargetrates)):
    zSpreadTmp = BisectionsSpread(zcbPriced, -0.2, 0.2, coupontargetrates[iCount],notionaltargetrates[iCount],zcbPriced.coupondatefrac[iCount],zcbPriced.notionaldatefrac[iCount],zcbPriced.couponcfs[iCount],zcbPriced.notionalcfs[iCount],zcbPriced.sspread[iCount],zcbPriced.marketprice[iCount], Precision, 1000)
    zcbPriced.zspread = np.append(zcbPriced.zspread,zSpreadTmp)

In [ ]:
zcbPriced.marketprice

array([94., 92.])

In [ ]:
coupontargetrates

[array([0.04474545, 0.04352242, 0.04174698, 0.04015462]),
 array([0.04474545, 0.04352242, 0.04174698, 0.04015462, 0.03903412,
        0.03830698])]

In [ ]:
zcbPriced.sspread

array([0.01, 0.01])

In [ ]:
zcbPriced.zspread

array([0.00056424, 0.02093499])